In [5]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

In [1]:
api_key = "83LYCH21FSJ4UQ61"

# List of top 10 NASDAQ stocks
nasdaq_stocks = ["AAPL", "MSFT", "AMZN", "GOOGL", "FB", "TSLA", "NVDA", "PYPL", "INTC", "CSCO"]

# Base URL for Alpha Vantage API
base_url = "https://www.alphavantage.co/query"


In [2]:
def fetch_month_data(symbol, year, month):
    print(f"Fetching data for {symbol} - {year}-{month:02d}...")

    # Define API parameters
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": "15min",
        "apikey": api_key,
        "outputsize": "full"  # Remove 'month' since it's not supported
    }

    # Send API request
    response = requests.get(base_url, params=params)

    # Check response
    if response.status_code == 200:
        try:
            # Parse JSON data returned by Alpha Vantage
            data = response.json()

            if "Time Series (15min)" in data:
                time_series = data["Time Series (15min)"]

                # Convert to DataFrame
                df = pd.DataFrame.from_dict(time_series, orient="index")
                df.columns = ["Open", "High", "Low", "Close", "Volume"]
                df.index = pd.to_datetime(df.index)
                df.sort_index(inplace=True)

                print(f"Fetched {len(df)} records for {symbol} - {year}-{month:02d}.")
                return df
            else:
                print(f"No time series data found for {symbol} - {year}-{month:02d}.")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error parsing data for {symbol} - {year}-{month:02d}: {e}")
            return pd.DataFrame()
    else:
        print(f"Failed to fetch data for {symbol} - {year}-{month:02d}. HTTP Status Code: {response.status_code}")
        return pd.DataFrame()


In [3]:
def fetch_stock_data(symbol):
    print(f"Starting data fetch for {symbol}...")

    # Initialize an empty DataFrame to hold the data
    all_data = pd.DataFrame()

    # Start from the most recent month and go backward until we get ~10,000 candles
    today = datetime.today()
    year = 2024
    month = 12
    total_records = 0

    while total_records < 10000:
        # Fetch data for the current month
        month_data = fetch_month_data(symbol, year, month)

        # Add to the combined data
        all_data = pd.concat([all_data, month_data], ignore_index=True)
        total_records += len(month_data)

        # Move to the previous month
        month -= 1
        if month == 0:
            month = 12
            year -= 1

        # Break the loop if no more data is available
        if month_data.empty:
            print(f"No more data available for {symbol}.")
            break

    # Save combined data to a CSV file
    filename = f"{symbol}_15min_candles.csv"
    all_data.to_csv(filename, index=False)
    print(f"Data for {symbol} saved to {filename} with {len(all_data)} records.")


In [6]:
for stock in nasdaq_stocks:
    fetch_stock_data(stock)
    print(f"Finished fetching data for {stock}.")

Starting data fetch for AAPL...
Fetching data for AAPL - 2024-12...
Fetched 1140 records for AAPL - 2024-12.
Fetching data for AAPL - 2024-11...
Fetched 1140 records for AAPL - 2024-11.
Fetching data for AAPL - 2024-10...
Fetched 1140 records for AAPL - 2024-10.
Fetching data for AAPL - 2024-09...
Fetched 1140 records for AAPL - 2024-09.
Fetching data for AAPL - 2024-08...
Fetched 1140 records for AAPL - 2024-08.
Fetching data for AAPL - 2024-07...
Fetched 1140 records for AAPL - 2024-07.
Fetching data for AAPL - 2024-06...
Fetched 1140 records for AAPL - 2024-06.
Fetching data for AAPL - 2024-05...
Fetched 1140 records for AAPL - 2024-05.
Fetching data for AAPL - 2024-04...
Fetched 1140 records for AAPL - 2024-04.
Data for AAPL saved to AAPL_15min_candles.csv with 10260 records.
Finished fetching data for AAPL.
Starting data fetch for MSFT...
Fetching data for MSFT - 2024-12...
Fetched 1140 records for MSFT - 2024-12.
Fetching data for MSFT - 2024-11...
Fetched 1140 records for MSFT -